In [ ]:
#'lines' is a list containing regions of image which could be lines

import cv2
import numpy as np
import os
import Line

def findLines(binary, path):
	kernelr1 = np.ones((1,7), np.uint8)

	#Pre-process input image
	eroded = cv2.erode(binary, kernelr1, iterations = 1)
	inverted = cv2.bitwise_not(eroded)

	#Scribbling board for identifying lines
	contours,hierarchy = cv2.findContours(inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	board1 = np.zeros(binary.shape, np.uint8)
	for contour in contours:
		if cv2.contourArea(contour)<150:
			continue
		hull = cv2.convexHull(contour)
		cv2.drawContours(board1, [hull], -1, 255, 2)
		x,y,w,h = cv2.boundingRect(contour)
		cv2.line(board1, (x-50,int(y+h/2)), (x+w+50,int(y+h/2)), 255, 5)

	#Extracting lines
	lines = []
	contours,hierarchy = cv2.findContours(board1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	for contour in contours:
		#Find minimum bounding rectangles(to obtain angle by thich it should be rotated)
		rect = cv2.minAreaRect(contour)
		box = cv2.boxPoints(rect)
		box = np.int0(box)

		#Extract line
		mask = np.zeros(binary.shape[:2], np.uint8)
		cv2.drawContours(mask, [contour], -1, 255, -1)
		line = cv2.bitwise_and(binary, binary, mask = mask)

		#Rotate line and mask
		extended = cv2.copyMakeBorder(line, 20,20,20,20, cv2.BORDER_CONSTANT, value = 0)
		extendedmask = cv2.copyMakeBorder(mask, 20,20,20,20, cv2.BORDER_CONSTANT, value = 0)
		rows,cols = extended.shape
		centerx, centery = rect[0]
		angle = rect[2] if rect[2]>=-45 else rect[2]+90
		M = cv2.getRotationMatrix2D((int(centerx)+20, int(centery)+20),angle,1)
		rotated = cv2.warpAffine(extended, M, (cols,rows))
		rotatedmask = cv2.warpAffine(extendedmask, M, (cols,rows))

		#Store line in lines[]
		contours,hierarchy = cv2.findContours(rotatedmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
		x,y,w,h = cv2.boundingRect(contours[0])
		#only the line, nothing above, nothing below
		extracted_region = rotated[y+3:y+h-3,x+3:x+w-3].copy()
		extracted_mask = rotatedmask[y+3:y+h-3,x+3:x+w-3].copy()
		extracted_mask_inv = cv2.bitwise_not(extracted_mask)
		extracted_line = cv2.bitwise_or(extracted_mask_inv, extracted_region)

		line_padded = cv2.copyMakeBorder(extracted_line, 10,10,10,10, cv2.BORDER_CONSTANT, value = 255)
		lines.insert(0, line_padded)

	#Make folders to save images and search for words
	i=1
	for line in lines:
		line_path = path + "/Line%d" %i
		os.mkdir(line_path)
		cv2.imwrite(line_path + "/line%d.jpg" %i, line)
		i+=1
		Line.findWords(line, line_path)